# Doing a regression on the California housing prices dataset
## Based on Chap 10 of Hands on ML 3rd ed

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

In [3]:
# get and split the data
X_trainval, X_test, y_trainval, y_test = train_test_split(housing.data, housing.target)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval)

In [5]:
# not saved as a df
X_train

array([[   3.9318    ,    7.        ,    5.3384785 , ...,    3.14994487,
          38.71      , -121.38      ],
       [   2.7222    ,   24.        ,    5.21279555, ...,    2.54102921,
          36.8       , -119.77      ],
       [   1.6654    ,   15.        ,    4.90625   , ...,    2.5703125 ,
          41.78      , -124.19      ],
       ...,
       [   3.3268    ,   23.        ,    4.44061962, ...,    2.36603557,
          37.47      , -122.22      ],
       [   3.6298    ,   34.        ,    4.28968254, ...,    5.25793651,
          34.06      , -117.8       ],
       [   4.4545    ,   21.        ,    5.8375    , ...,    3.1578125 ,
          34.02      , -117.49      ]])

In [7]:
# these are the feature names
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [8]:
#scale the variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## Use sequential API to build model, similar to classification, except with a single output neuron

In [9]:
import tensorflow as tf
from tensorflow import keras

C:\anaconda_python\envs\insight_2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\anaconda_python\envs\insight_2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
tf.__version__

'2.0.0'

In [11]:
# Don't need a flatten in this case
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])

In [12]:
model.compile(loss = 'mean_squared_error', optimizer='sgd')

In [14]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 104us/sample - loss: 1.4445 - val_loss: 13.6939
Epoch 2/20
11610/11610 [==============================] - 1s 49us/sample - loss: 1.3761 - val_loss: 12.5552
Epoch 3/20
11610/11610 [==============================] - 1s 51us/sample - loss: 0.9289 - val_loss: 0.4354
Epoch 4/20
11610/11610 [==============================] - 1s 50us/sample - loss: 0.3964 - val_loss: 0.3892
Epoch 5/20
11610/11610 [==============================] - 1s 54us/sample - loss: 0.3746 - val_loss: 0.3768
Epoch 6/20
11610/11610 [==============================] - 1s 60us/sample - loss: 0.3631 - val_loss: 0.3730
Epoch 7/20
11610/11610 [==============================] - 1s 47us/sample - loss: 0.3584 - val_loss: 0.3684
Epoch 8/20
11610/11610 [==============================] - 1s 57us/sample - loss: 0.3520 - val_loss: 0.3684
Epoch 9/20
11610/11610 [==============================] - 1s 51us/sample - loss: 0.3479 - va

In [15]:
mse_test = model.evaluate(X_test, y_test)

5160/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

### Sequential API is easy to use, but sometimes we may want more complex models

## Using the Functional API

### For example build a wide and deep NN. Learn both deep patterns as well as simple rules

In [20]:
# input object, specifies the kind of input the model will get, can have multiple
input_ = keras.layers.Input(shape=X_train.shape[1:])
# create a dense layer and call like a function, passing the input (functional part of API)
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)

#create a concatenate layer and join input and output of second hidden layer
concat = keras.layers.concatenate([input_, hidden2])
#output layer with single neuron and no activation fn
output = keras.layers.Dense(1)(concat)

model = keras.Model(inputs=[input_], outputs=[output])

In [21]:
model.compile(loss ='mse', optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 81us/sample - loss: 2.8173 - val_loss: 1.8804
Epoch 2/20
11610/11610 [==============================] - 1s 53us/sample - loss: 0.7343 - val_loss: 0.6670
Epoch 3/20
11610/11610 [==============================] - 1s 52us/sample - loss: 0.5895 - val_loss: 0.5867
Epoch 4/20
11610/11610 [==============================] - 1s 54us/sample - loss: 0.5650 - val_loss: 0.5650
Epoch 5/20
11610/11610 [==============================] - 1s 56us/sample - loss: 0.5456 - val_loss: 0.5500
Epoch 6/20
11610/11610 [==============================] - 1s 54us/sample - loss: 0.5307 - val_loss: 0.5354
Epoch 7/20
11610/11610 [==============================] - 1s 52us/sample - loss: 0.5177 - val_loss: 0.5221
Epoch 8/20
11610/11610 [==============================] - 1s 55us/sample - loss: 0.5064 - val_loss: 0.5124
Epoch 9/20
11610/11610 [==============================] - 1s 51us/sample - loss: 0.4965 - val_l

## Try sending a subset of features through wide path and different subset (possibly overlapping) through deep path

In [22]:
# input object, specifies the kind of input the model will get, can have multiple
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
# create a dense layer and call like a function, passing the input (functional part of API)
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)

#create a concatenate layer and join input and output of second hidden layer
concat = keras.layers.concatenate([input_A, hidden2])
#output layer with single neuron and no activation fn
output = keras.layers.Dense(1, name='output')(concat)

model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [23]:
model.compile(loss ='mse', optimizer=keras.optimizers.SGD(lr=1e-3))

In [24]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_val_A, X_val_B = X_val[:, :5], X_val[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

In [25]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_val_A, X_val_B), y_val))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 81us/sample - loss: 2.1860 - val_loss: 1.4624
Epoch 2/20
11610/11610 [==============================] - 1s 58us/sample - loss: 0.7699 - val_loss: 0.6862
Epoch 3/20
11610/11610 [==============================] - 1s 54us/sample - loss: 0.6233 - val_loss: 0.6195
Epoch 4/20
11610/11610 [==============================] - 1s 52us/sample - loss: 0.5880 - val_loss: 0.5932
Epoch 5/20
11610/11610 [==============================] - 1s 53us/sample - loss: 0.5662 - val_loss: 0.5735
Epoch 6/20
11610/11610 [==============================] - 1s 56us/sample - loss: 0.5493 - val_loss: 0.5571
Epoch 7/20
11610/11610 [==============================] - 1s 55us/sample - loss: 0.5352 - val_loss: 0.5428
Epoch 8/20
11610/11610 [==============================] - 1s 57us/sample - loss: 0.5236 - val_loss: 0.5312
Epoch 9/20
11610/11610 [==============================] - 1s 53us/sample - loss: 0.5132 - val_l

In [26]:
mse_test = model.evaluate((X_test_A, X_test_B), y_test)

5160/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [27]:
mse_test

0.4473105427830718

## Saving a model

In [30]:
# saves in hdf5 - model architecture, parameters, optimizer etc
model.save('../models/model1.h5')

## And to load it

In [31]:
model = keras.models.load_model('../models/model1.h5')

## Using callbacks

1. list of objects keras will call at the start and end of training at the start and end of each epoch
2. ModelCheckpoint can be used to save checkpoints at regular intervals during training, by default at the end of each epoch
3. Can also set a save_best_only=True which saves only when performance on validation set is best so far

In [32]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('../models/model_checkpoint.h5', save_best_only=True)
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_val_A, X_val_B), y_val), callbacks=[checkpoint_cb])
# roll back to best model
model = keras.models.load_model('../models/model_checkpoint.h5')

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 2s 188us/sample - loss: 0.4536 - val_loss: 0.4572
Epoch 2/20
11610/11610 [==============================] - 1s 67us/sample - loss: 0.4509 - val_loss: 0.4538
Epoch 3/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.4484 - val_loss: 0.4510
Epoch 4/20
11610/11610 [==============================] - 1s 75us/sample - loss: 0.4464 - val_loss: 0.4494
Epoch 5/20
11610/11610 [==============================] - 1s 67us/sample - loss: 0.4442 - val_loss: 0.4466
Epoch 6/20
11610/11610 [==============================] - 1s 62us/sample - loss: 0.4423 - val_loss: 0.4441
Epoch 7/20
11610/11610 [==============================] - 1s 62us/sample - loss: 0.4404 - val_loss: 0.4422
Epoch 8/20
11610/11610 [==============================] - 1s 65us/sample - loss: 0.4385 - val_loss: 0.4400
Epoch 9/20
11610/11610 [==============================] - 1s 67us/sample - loss: 0.4366 - val_

### Can also do this using Early Stopping checkpoint

In [34]:
# Didn't actually trigger here but would stop early if the loss stopped decreasing/increased
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit((X_train_A, X_train_B), y_train, epochs=100, validation_data=((X_val_A, X_val_B), y_val), callbacks=[checkpoint_cb, early_stopping_cb])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 62us/sample - loss: 0.4000 - val_loss: 0.4018
Epoch 2/100
11610/11610 [==============================] - 1s 61us/sample - loss: 0.3991 - val_loss: 0.4021
Epoch 3/100
11610/11610 [==============================] - 1s 60us/sample - loss: 0.3981 - val_loss: 0.4018
Epoch 4/100
11610/11610 [==============================] - 1s 60us/sample - loss: 0.3974 - val_loss: 0.3994
Epoch 5/100
11610/11610 [==============================] - 1s 65us/sample - loss: 0.3962 - val_loss: 0.4017
Epoch 6/100
11610/11610 [==============================] - 1s 74us/sample - loss: 0.3953 - val_loss: 0.3980
Epoch 7/100
11610/11610 [==============================] - 1s 68us/sample - loss: 0.3942 - val_loss: 0.3991
Epoch 8/100
11610/11610 [==============================] - 1s 64us/sample - loss: 0.3935 - val_loss: 0.3962
Epoch 9/100
11610/11610 [==============================] - 1s 60us/sample - loss: 0.392

## Using tensorboard

In [43]:
import os
root_logdir = '..\\logs\\'

In [44]:
import time
def get_run_logdir():
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

In [45]:
run_logdir=get_run_logdir()

In [46]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_val_A, X_val_B), y_val), callbacks=[tensorboard_cb])

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.3371 - val_loss: 0.3474
Epoch 2/20
11610/11610 [==============================] - 1s 61us/sample - loss: 0.3372 - val_loss: 0.3497
Epoch 3/20
11610/11610 [==============================] - 1s 63us/sample - loss: 0.3367 - val_loss: 0.3461
Epoch 4/20
11610/11610 [==============================] - 1s 67us/sample - loss: 0.3365 - val_loss: 0.3488
Epoch 5/20
11610/11610 [==============================] - 1s 63us/sample - loss: 0.3363 - val_loss: 0.3471
Epoch 6/20
11610/11610 [==============================] - 1s 120us/sample - loss: 0.3357 - val_loss: 0.3466
Epoch 7/20
11610/11610 [==============================] - 1s 73us/sample - loss: 0.3354 - val_loss: 0.3470
Epoch 8/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.3352 - val_loss: 0.3453
Epoch 9/20
11610/11610 [==============================] - 1s 79us/sample - loss: 0.3350 - val_

In [ ]:
!tensorboard --logdir=root_logdir --port=6006